In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from collections import defaultdict
from urllib.parse import urlparse


# website request
base_url = "https://www.haberturk.com/"
response = requests.get(base_url)

def get_domain(url):
    # Parse the URL and extract the domain
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    
    # Remove 'www.' if present
    if domain.startswith('www.'):
        domain = domain[4:]
    
    return domain

naked_domain = get_domain(base_url)
print(naked_domain)

# parse HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# URL lists
url_list = []
external_urls = []
segment_list = defaultdict(list)
internal_urls = []
absolute_urls = []
unique_external_urls = []
unique_urls = []
relative_urls = []

# get href values
for tag in soup.find_all(href=True):
    href = tag['href']
    url_list.append(href)

# find unique URLs
for url in url_list:
    if url not in unique_urls:
        unique_urls.append(url)

# find URLs
for url in url_list:
     if url not in internal_urls:
        if naked_domain in url or url.startswith('/'):
            internal_urls.append(url)
            if url.startswith('http'):
                absolute_urls.append(url)
            else:
                relative_urls.append(url)
        elif naked_domain not in url:
            external_urls.append(url)
            for url in external_urls:
                if url not in unique_external_urls:
                    unique_external_urls.append(url)
            

# extract the first important path segment
def get_first_segment(url):
    segments = url.strip('/').split('/')
    if segments:
        return segments[0]
    return None

# similar and unique categorization
for url in url_list:
    if url.startswith('/'):  # only relative URLs
        segment = get_first_segment(url)
        if segment:
            segment_list[segment].append(url)


# save to JSON
result = {
    "internal_urls": list(internal_urls),
    "relative_urls": list(relative_urls),
    "external_urls": list(external_urls),
    "absolute_urls": list(absolute_urls),
    "unique_external_urls": list(unique_external_urls),
    "total_url_list": list(url_list),
    "unique_urls": list(unique_urls)
}
with open('urls.json', 'w') as json_file:
    json.dump(result, json_file, indent=4)
    print("Data has been saved to JSON")

haberturk.com
Data has been saved to JSON
